# Buidling Causal Graphical Models

Reminder that we're using tools to infer what the data generating process (DGP) may look like. One set of data may imply many different DGPs, so it's a hard game to play. This chapter is focused on graphically representing the DGP.

## 3.1.1 Transportation Case Study

In [5]:
# 3.2 Building the transportation DAG in pgmpy

from pgmpy.models import DiscreteBayesianNetwork
model = DiscreteBayesianNetwork(
    [
        ('Age', 'Education'),
        ('Gender', 'Education'),
        ('Education', 'Occupation'),
        ('Education', 'Residence'),
        ('Occupation', 'Transportation'),
        ('Residence', 'Transportation')
    ]
)

model

## 3.1.3 DAGs as Communication Tools

- Examples in which you've used DAGs for communication
- Discuss DAG limitations as a reminder
    - "How" vs "What"
- Logic Gates visualize "How"
    

DAGs represent and codify causal assumptions that can be used in further computation purposes. they also represent time, in that the causal arrow assumes forward movement in time. 

Identified ways of getting around the "acyclic" assumption in DAGs without necessarily having to relax the criteria. Example given is to unpack the "cycles" into discrete steps and track their path over time. 

## Linking Causality to Conditional Independence

Highlighted how causal DAG allows a much simpler structure for formulating joint probability, because the causal pathway carries some assumptions about conditional independence.

Specifically, in the geneology example, the author demonstrates where the 2 parents' blood types contain enough info to explain a child's blood type without knowing the parents' blood type.

This characteristic, where direct "parent" properties supercede "grandparent" properties, is called the "causal Markov property"



## Scaffolding for Causal Machine Learning Models

Building with the DAG as the scaffolding permits the further goal - building causal machine learning models. These can be used for prediction and causal inference.

After factoring under the assumption of causal conditional indepdence, the "factors" are also called Markov Kernels



### Labeling Causal Abstractions

Labels are by their nature fluid and changing, so it's important to be careful about consistency among labels. Author citied the example of race, whose definition has definitely changed over time.

"In machine learning, we're often encouraged to blindly label data and not think about the DGP"



## Training a Model on a Causal Dag

In [19]:
import pandas as pd
url = 'https://raw.githubusercontent.com/altdeep/causalML/master/datasets/transportation_survey.csv'
df = pd.read_csv(url)
df.columns = ["Age", "Gender", "Education", "Occupation", "Residence", "Transportation"]

In [20]:
# There are people in this world who use data without looking at it. Don't let it be you!
# in notebooks, "sample" can give you a better idea what's in a dataframe than "head"

df.sample(10)

,Age,Gender,Education,Occupation,Residence,Transportation
185,adult,F,uni,emp,big,other
490,adult,F,uni,emp,big,car
407,young,F,uni,emp,big,car
461,adult,F,uni,emp,big,car
54,old,M,high,emp,big,train
149,young,M,high,emp,big,train
310,adult,F,high,emp,big,car
138,young,F,uni,emp,big,other
74,adult,F,uni,emp,big,car
145,young,M,uni,emp,big,other


In [21]:
# Learning Parameters for the causal Markov kernels
model.fit(df)  # polars wins on convenience, pandas wins on compatibility
causal_markov_kernels = model.get_cpds()
print(causal_markov_kernels)


INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'Age': 'C', 'Gender': 'C', 'Education': 'C', 'Occupation': 'C', 'Residence': 'C', 'Transportation': 'C'}


[<TabularCPD representing P(Age:3) at 0x3012d6c00>, <TabularCPD representing P(Education:2 | Age:3, Gender:2) at 0x302d3a0c0>, <TabularCPD representing P(Gender:2) at 0x3017e0650>, <TabularCPD representing P(Occupation:2 | Education:2) at 0x302d1e9c0>, <TabularCPD representing P(Residence:2 | Education:2) at 0x302d1f6b0>, <TabularCPD representing P(Transportation:3 | Occupation:2, Residence:2) at 0x302d1f9b0>]


In [ ]:
# AI Generated code block in case you're having trouble getting the full visual in the book
import pandas as pd
import numpy as np

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

cmk_T = causal_markov_kernels[-1]

print(f"Variable: {cmk_T.variable}")
print(f"Evidence variables: {cmk_T.variables[1:]}")
print(f"State names: {cmk_T.state_names}")
print(f"\nValues shape: {cmk_T.values.shape}")
print(f"\nFull conditional probability table:")

# Reshape the 3D array into 2D for better display
# Shape is (3 transportation types, 2 occupations, 2 residences)
values = cmk_T.values
n_transport, n_occup, n_resid = values.shape

# Create column labels for each combination of Occupation and Residence
columns = []
for i in range(n_occup):
    for j in range(n_resid):
        columns.append(f"Occ={cmk_T.state_names['Occupation'][i]}, Res={cmk_T.state_names['Residence'][j]}")

# Reshape to 2D: rows are transportation types, columns are combinations of evidence
values_2d = values.reshape(n_transport, -1)
df_cpd = pd.DataFrame(
    values_2d,
    index=[f"Transportation={t}" for t in cmk_T.state_names['Transportation']],
    columns=columns
)

print(df_cpd)

Variable: Transportation
Evidence variables: ['Occupation', 'Residence']
State names: {'Transportation': ['car', 'other', 'train'], 'Occupation': ['emp', 'self'], 'Residence': ['big', 'small']}

Values shape: (3, 2, 2)

Full conditional probability table:
                      Occ=emp, Res=big  Occ=emp, Res=small  Occ=self, Res=big  \
Transportation=car            0.703431            0.524390           0.444444   
Transportation=other          0.134804            0.085366           0.333333   
Transportation=train          0.161765            0.390244           0.222222   

                      Occ=self, Res=small  
Transportation=car                    1.0  
Transportation=other                  0.0  
Transportation=train                  0.0  
